# Create a sequence from scratch

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import georges_core
from georges_core.sequences import PlacementSequence, SequenceMetadata, SurveySequence, Element
from georges_core.units import ureg as _ureg
from georges_core import Kinematics

import georges
from georges import vis
from georges import manzoni
from georges.manzoni import Input, Beam
from georges.manzoni.integrators import MadXIntegrator, \
                    TransportFirstOrderTaylorIntegrator, \
                    TransportSecondOrderTaylorIntegrator, \
                    TransportFirstOrderTaylorIntegratorExact, \
                    TransportSecondOrderTaylorIntegratorExact

In [ ]:
def set_size(width, fraction=1, subplots=(1, 1)):
    """ Set figure dimensions to avoid scaling in LaTeX.

     Parameters
    ----------
    width: float or string
    Document width in points, or string of predined document type
    fraction: float, optional
    Fraction of the width which you wish the figure to occupy
    subplots: array-like, optional
    The number of rows and columns of subplots.
    Returns
    -------
    fig_dim: tuple
    Dimensions of figure in inches
    """
    if width == 'thesis':
        width_pt = 426.79135
    elif width == 'beamer':
        width_pt = 307.28987
    elif width == 'pnas':
        width_pt = 246.09686
    else:
        width_pt = width

     # Width of figure (in pts)
    fig_width_pt = width_pt * fraction
    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

     # Golden ratio to set aesthetic figure height
    golden_ratio = (5**.5 - 1) / 2

     # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio * (subplots[0] / subplots[1])

    return (fig_width_in, fig_height_in)

def get_paper_figure_parameter(width, height=None, fontsize=10, labelsize=8, **kwargs):

    if height == None:
        plt.rcParams['figure.figsize'] = set_size(width)
    else:
        plt.rcParams['figure.figsize'] = (width * 1 / 72.27, height * 1 / 72.27)

    plt.rc('text', usetex=True)
    plt.rc('font', family='serif', serif='Times', size=fontsize)
    plt.rc('xtick', labelsize=labelsize)
    plt.rc('ytick', labelsize=labelsize)
    plt.rc('axes', labelsize=labelsize)

     # plt.rcParams['axes.labelsize'] = 10
    plt.rcParams['lines.markersize'] = 5
    plt.rcParams['lines.linewidth'] = 0.7
    plt.rcParams['legend.fontsize'] = 8

     ## Figure parameters
    plt.rcParams['figure.dpi'] = kwargs.get('dpi', 500)
    plt.rcParams['figure.facecolor'] = 'w'
    plt.rcParams['figure.edgecolor'] = 'k'
    plt.rcParams['figure.autolayout'] = True
    plt.rcParams['savefig.dpi'] = kwargs.get('dpi', 500)

     ## Legend sizes
    plt.rcParams['axes.grid'] = True
    plt.rcParams['axes.linewidth'] = 0.1 #set the value globally

    return plt.rc

### Define kinematics and beam

In [ ]:
particle = georges_core.particles.Proton
kin = georges_core.Kinematics(230*_ureg.MeV,
                              particle=particle, 
                              kinetic=True)

In [ ]:
beam_distr = georges_core.Distribution().from_5d_multigaussian_distribution(n=10000,
                                                                            XRMS=5e-3,
                                                                            PXRMS=5e-3,
                                                                            YRMS=5e-3,
                                                                            PYRMS=5e-3).distribution
beam = Beam(kinematics = kin,
            distribution = beam_distr)

### Define sequence manually

In [ ]:
def define_sequence(integrator, aperture): # Generic function to modify the manzoni integrator and the aperture
    
    d1 = Element.Drift(NAME="D1",
                   integrator=integrator,
                   L=0.3* _ureg.m,
                   APERTYPE="CIRCULAR",
                   APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    qf = Element.Quadrupole(NAME="Q1",
                            integrator=integrator,
                            L=0.3*_ureg.m,
                            K1=2*_ureg.m**-2,
                            APERTYPE="CIRCULAR",
                            APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    d2 = Element.Drift(NAME="D2",
                       integrator=integrator,
                       L=0.3*_ureg.m,
                       APERTYPE="CIRCULAR",
                       APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    b1 = Element.SBend(NAME="B1",
                       integrator=integrator,
                       L=1*_ureg.m,
                       ANGLE=30*_ureg.degrees,
                       K1=0*_ureg.m**-2,
                       APERTYPE="CIRCULAR",
                       APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    d3 = Element.Drift(NAME="D3",
                       integrator=integrator,
                       L=0.3*_ureg.m,
                       APERTYPE="CIRCULAR",
                       APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    qd = Element.Quadrupole(NAME="Q2",
                            integrator=integrator,
                            L=0.3*_ureg.m,
                            K1=-2*_ureg.m**-2,
                            APERTYPE="CIRCULAR",
                            APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    d4 = Element.Drift(NAME="D4",
                       integrator=integrator,
                       L=0.3*_ureg.m,
                       APERTYPE="CIRCULAR",
                       APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    b2 = Element.SBend(NAME="B2",
                       integrator=integrator,
                       L=1*_ureg.m,
                       ANGLE=-30*_ureg.degrees,
                       K1=0*_ureg.m**-2,
                       APERTYPE="CIRCULAR",
                       APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    d5 = Element.Drift(NAME="D5",
                       integrator=integrator,
                       L=0.3*_ureg.m,
                       APERTYPE="CIRCULAR",
                       APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    
    sequence = PlacementSequence(name="TEST")
    
    sequence.place(d1,at_entry=0)
    sequence.place_after_last(qf)
    sequence.place_after_last(d2)
    sequence.place_after_last(b1)
    sequence.place_after_last(d3)
    sequence.place_after_last(qd)
    sequence.place_after_last(d4)
    sequence.place_after_last(b2)
    sequence.place_after_last(d5)    

    return sequence

In [ ]:
sequence_madx = define_sequence(MadXIntegrator, 10)
sequence_transport_order_1 = define_sequence(TransportFirstOrderTaylorIntegrator, 10)
sequence_transport_order_2 = define_sequence(TransportSecondOrderTaylorIntegrator, 10)

### Load sequence using SurveySequence (from a csv file)

#### Based on AT

In [ ]:
sequence_survey = SurveySequence(filename="survey.csv")
sequence_survey.expand(); # This must be done before tracking !

#### Based on XYZ

In [ ]:
sequence_survey_global = SurveySequence(filename="bdsim_survey.dat")

### Track with MANZONI

In [ ]:
mi_madx = manzoni.Input.from_sequence(sequence=sequence_madx)
mi_transport_order_1 = manzoni.Input.from_sequence(sequence=sequence_transport_order_1)
mi_transport_order_2 = manzoni.Input.from_sequence(sequence=sequence_transport_order_2)
mi_survey = manzoni.Input.from_sequence(sequence=sequence_survey)

obs_madx = georges.manzoni.SigmaObserver()
obs_transport_order_1 = georges.manzoni.SigmaObserver()
obs_transport_order_2 = georges.manzoni.SigmaObserver()
obs_survey = georges.manzoni.SigmaObserver()

mi_madx.track(beam=beam,
              observers=[obs_madx])
mi_transport_order_1.track(beam=beam,
              observers=[obs_transport_order_1])
mi_transport_order_2.track(beam=beam,
              observers=[obs_transport_order_2])
mi_survey.track(beam=beam,
              observers=[obs_survey])

res_madx = obs_madx.to_df()
res_transport_order_1 = obs_transport_order_1.to_df()
res_transport_order_2 = obs_transport_order_2.to_df()
res_survey = obs_survey.to_df()

In [ ]:
res_survey

In [ ]:
res_survey.set_index("LABEL1",inplace=True)
res_survey.drop(index=['START','END'],inplace=True)

In [ ]:
res_survey

### Load BDSIM results

In [ ]:
bdsim_res = pd.read_csv('bdsim_results.csv')

In [ ]:
bdsim_res

### Plots for comparison

In [ ]:
df = sequence_madx.df.copy()
for line,_ in df.iterrows():
    if line != "D1":
        df.at[line,'AT_ENTRY'] = df.at[line,'AT_ENTRY'].m_as('m')
        df.at[line,'AT_CENTER'] = df.at[line,'AT_CENTER'].m_as('m')
        df.at[line,'AT_EXIT'] = df.at[line,'AT_EXIT'].m_as('m')
        
df.at['D1','AT_CENTER'] = df.at['D1','AT_CENTER'].m_as('m')
df.at['D1','AT_EXIT'] = df.at['D1','AT_EXIT'].m_as('m')

df['TYPE'] = ['Drift','Quadrupole','Drift','SBend','Drift','Quadrupole','Drift','SBend','Drift']
df['CLASS'] = ['Drift','Quadrupole','Drift','SBend','Drift','Quadrupole','Drift','SBend','Drift']

### X axis

In [ ]:
plt.rc=get_paper_figure_parameter(238.25444)
plt.rcParams['legend.fontsize'] = 10
plt.rc('text', usetex=True)

# fig = plt.figure()
# ax = fig.add_subplot(111)

# manzoni_plot = vis.ManzoniMatplotlibArtist(ax=ax)

# manzoni_plot.prepare(ax,
#                      df,
#                      with_beamline=True,
#                      print_label=False,
#                      ylim=[-20,20])

plt.plot(bdsim_res['S'],
         1e3*res_madx['BEAM_OUT_X'],
         color='b',
         linestyle='dashed',
         marker='*',
         label="Manzoni - MadX",
         markersize=2)
plt.plot(bdsim_res['S'],
         1e3*res_transport_order_1['BEAM_OUT_X'],
         color='k',
         linestyle='dashed',
         marker='*',
         label="Manzoni - Transport 1",
         markersize=2)
plt.plot(bdsim_res['S'],
         1e3*res_transport_order_2['BEAM_OUT_X'],
         color='g',
         linestyle='dashed',
         marker='*',
         label="Manzoni - Transport 2",
         markersize=2)
plt.plot(bdsim_res['S'],
         1e3*res_survey['BEAM_OUT_X'],
         color='m',
         linestyle='dashed',
         marker='*',
         label="Manzoni - MadX - from survey",
         markersize=2)

plt.plot(bdsim_res['S'],
         1e3*bdsim_res['X'],
         color='r',
         linestyle='dashed',
         marker='*',
         label='BDSIM',
         markersize=2)

plt.xlabel('S (m)')
plt.ylabel('$\sigma_x$ (mm)')

plt.grid()
plt.legend(fontsize=6)

### Y axis

In [ ]:
plt.rc=get_paper_figure_parameter(238.25444)
plt.rcParams['legend.fontsize'] = 10
plt.rc('text', usetex=True)

plt.plot(bdsim_res['S'],
         1e3*res_madx['BEAM_OUT_Y'],
         color='b',
         linestyle='dashed',
         marker='*',
         label="Manzoni - MadX",
         markersize=2)
plt.plot(bdsim_res['S'],
         1e3*res_transport_order_1['BEAM_OUT_Y'],
         color='k',
         linestyle='dashed',
         marker='*',
         label="Manzoni - Transport 1",
         markersize=2)
plt.plot(bdsim_res['S'],
         1e3*res_transport_order_2['BEAM_OUT_Y'],
         color='g',
         linestyle='dashed',
         marker='*',
         label="Manzoni - Transport 2",
         markersize=2)
plt.plot(bdsim_res['S'],
         1e3*res_survey['BEAM_OUT_Y'],
         color='m',
         linestyle='dashed',
         marker='*',
         label="Manzoni - MadX - from survey",
         markersize=2)

plt.plot(bdsim_res['S'],
         1e3*bdsim_res['Y'],
         color='r',
         linestyle='dashed',
         marker='*',
         label='BDSIM',
         markersize=2)


plt.xlabel('S (m)')
plt.ylabel('$\sigma_y$ (mm)')
plt.grid()
plt.legend(fontsize=6)